In [ ]:
# imports
import os
import sys
import time
import traceback
import threading

from qumulo.rest_client import RestClient

### Setup credentials, create the parent directory

In [ ]:
rc = RestClient("product", 8000)
rc.login("admin", "admin")

def create_dir(rc, name, dir_path='/'):
    try:
        rc.fs.create_directory(name = name, dir_path = dir_path)
    except:
        exc_type, exc_value, exc_traceback = sys.exc_info()
        print("Exception: %s" % exc_value)

# Create base user directory, if it doesn't already exist
create_dir(rc, name='users', dir_path='/')
create_dir(rc, name='dept', dir_path='/users')

### Create 5,000 Quotas

In [ ]:
threads = []
thread_count = 20
dir_count = 5000

def create_quotas(start_dir, end_dir):
    rc = RestClient("product", 8000)
    rc.login("admin", "admin")
    for dir_num in range(start_dir, end_dir):
        dir_name = 'u-' + str(dir_num)
        try:
            dir_res = rc.fs.get_attr(path='/users/dept/' + dir_name)
        except:
            dir_res = rc.fs.create_directory(name=dir_name, dir_path='/users/dept')
        dir_id = dir_res['file_number']
        quota_res = rc.quota.create_quota(id_ = dir_id, limit_in_bytes = 1000000000)

        
for proc_num in range(0, thread_count):
    start_dir = proc_num*dir_count / thread_count
    end_dir = (proc_num+1)*dir_count / thread_count
    t = threading.Thread(target=create_quotas, args=(start_dir, end_dir))
    threads.append(t)
    t.start()

# Check aggregates to see when all directories and quotas exist
while int(rc.fs.read_dir_aggregates(path='/users/dept/')['total_directories']) < 5000:
    print(int(rc.fs.read_dir_aggregates(path='/users/dept/')['total_directories']))
    time.sleep(2.5)

print("Done!")

### Enumerate all quotas and show a specific one

In [ ]:
for d in rc.quota.get_all_quotas_with_status():
    for q in d['quotas']:
        if q['path'] == '/users/dept/u-238/':
            print q

In [ ]:
rc.quota.get_quota_with_status(148001821)

### Create and write a 1 GB file

In [ ]:
new_file = rc.fs.create_file(name="1GB.txt", dir_path="/users/dept/u-238")

In [ ]:
import StringIO
f = StringIO.StringIO()
f.write('0'*1000000000)
written_file = rc.fs.write_file(data_file = f, path=new_file['path'])

### Can we still write to this directory?

In [ ]:
new_file = rc.fs.create_file(name="just-one-more---please.txt", dir_path="/users/dept/u-238")

### Delete all quotas

In [ ]:
d = rc.quota.get_all_quotas_with_status()
for q in d:
    for quota in q['quotas']:
        # but only delete them if they are in /users/dept
        if '/users/dept' in quota['path']:
            rc.quota.delete_quota(id_ = quota['id'])

### Tree delete all of the user directories and check if it's gone

In [ ]:
rc.fs.delete_tree(path='/users/dept/')

In [ ]:
rc.fs.read_directory(path='/users/')